<a href="https://colab.research.google.com/github/chodizzle/portfolio1/blob/main/tfidf_cats_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install python-Levenshtein
!pip install fuzzywuzzy

     |████████████████████████████████| 51kB 4.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149834 sha256=8c0c038eccab708b583e03ce5d6a8d493ae309a68ba6666004abff1137a4839b
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import Counter

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
df = pd.read_pickle('meats_v2.pickle')
# pickles are good mmkay
df.head()

,name,category,sw,distributor
item_id,,,,
8566653,beef brisket ch nat,Brisket,"[brisket, beef]",Del Monte Meat Company
3869441,"lamb, leg brt dmstc raw ref",Lamb Leg,"[lamb, leg]",US Foods
4626190,pork loin lonzetta rsm,Pork Loin,"[pork, loin]",PARAGON FOODS
3195109,hens rock cornish fresh,Top Sirloin,"[top, sirloin, beef]",KINNEALEY QUALITY MEATS
7756436,striploin 0x1 choice beef striploin usda choice,Beef Tenderloin,"[tenderloin, beef]",HALPERNS STEAK & SEAFOOD


In [ ]:
test_cats = pd.read_csv('test_cats.csv', index_col=0)
test_cats = test_cats['category'].values
test_cats

array(['Ground Beef', 'Top Round', 'Hot Dog', 'Beef Strip Steak',
       'New York Strip', 'Flat Iron Steak', 'Skirt Steak', 'Flap Meat',
       'Ground Chuck', 'Wild Boar', 'Top Sirloin', 'Short Ribs',
       'Hanger Steak', 'Chuck Roast', 'Filet Mignon', 'Tri Tip',
       'Pork Rack', 'Chuck Roll', 'Prime Rib', 'Baby Back Ribs',
       'Pork Head', 'Bone Marrow', 'T-Bone Steaks', 'Chuck Flat',
       'Spare Ribs', 'Hot Link', 'Ham Hocks', 'Pork Skin', 'Pork Chops',
       'Pork Neck Bones', 'Wagyu Beef Rib Eye', 'Ground Kobe Beef',
       'Lamb', 'Cornish Game Hens', 'Pork Chop Center Cut'], dtype=object)

In [ ]:
# do the same as above but for every cat in test_cats
df_docs = pd.DataFrame(columns=['category', 'sw', 'doc'])

for cat in test_cats: 
  df_temp = df[df['category'] == cat]
  doc = ' '.join(df_temp['name'].values)
  sw = df_temp.iloc[0,2]
  doc_row = [{'category': cat, 'sw' : sw, 'doc': doc}]
  df_docs = df_docs.append(doc_row, ignore_index=True)

df_docs.set_index('category', inplace=True)
df_docs.head()

,sw,doc
category,,
Ground Beef,"[ground, beef]",cab beef grnd chck fine 80/20 f ground whit op...
Top Round,"[round, top, beef]",96oz black round shallow bowl bbrlcls beef cor...
Hot Dog,"[hot, dog, sausages]",vital fiber 7.76oz 10oz vegware compostable co...
Beef Strip Steak,"[strip, beef, steak]",beef strip loin 0x1 cho 180 beef gr pty 3-1 av...
New York Strip,"[new, strip, beef, york]",installed new receptacles for cannons above ba...


In [ ]:
# separate df to play around with stopwords
df_sw = df_docs['sw']
df_sw

category
Ground Beef                               [ground, beef]
Top Round                             [round, top, beef]
Hot Dog                             [hot, dog, sausages]
Beef Strip Steak                    [strip, beef, steak]
New York Strip                  [new, strip, beef, york]
Flat Iron Steak                [iron, steak, beef, flat]
Skirt Steak                         [skirt, beef, steak]
Flap Meat                             [flap, meat, beef]
Ground Chuck                       [ground, beef, chuck]
Wild Boar                                   [wild, boar]
Top Sirloin                         [top, sirloin, beef]
Short Ribs                           [short, beef, ribs]
Hanger Steak                       [hanger, beef, steak]
Chuck Roast                         [roast, beef, chuck]
Filet Mignon                       [mignon, beef, filet]
Tri Tip                                 [tip, tri, beef]
Pork Rack                             [pork, rack, ribs]
Chuck Roll            

TFIDF Vectorizer on df_docs!!!


Testing some preprocessing
1. Preprocess df_docs['doc'] with preprocessor to get rid of bad symbols (keep good ones)
2. Generate better stopwords using df_docs['sw']
3. ***Remove stopwords from df_docs['doc'] is fuzz.ratio > .8??***
    
    ***holding on fuzz as it might remove too many words from docs***

4. Rerun TFIDF

Also try:
1. Generate stopwords using df_docs['sw'] across all cats
2. Run TFIDF using my_preprocessor and comprehensive stopwords from 1.

Compare results.

**METHOD 1 IS BEST**

<h1>First Method: Stopwords by Category Then Vectorize


<h2>Preprocessor and Stopwords generator</h2>

In [ ]:
def stopword_gen(stopwords):
  # sw generator
  new_sw = []
  new_sw = new_sw + stopwords #populate with base stopwords
  # print(stopwords)
  for word in stopwords:
    # new stopwords with all vowels removed
    if not re.match('^[aeiou].*|.*[aeiou]$', word) and len(word) > 3:
      no_v_word = re.sub('[aeiou]', '', word)
      new_sw.append(no_v_word)

    # first and last letter
    if not re.match('^[aeiou].*|.*[aeiou]$|^l.*b$|^k.*g$|^o.*z$|^g.*m$', word) and len(word) > 3:
      short_word = word[0] + word[-1]
      new_sw.append(short_word)

    # special for chicken
    if word == 'chicken': new_sw + ['chix', 'chx']

  return '\s*'+'\s*|\s*'.join(set(new_sw))+'\s*'

In [ ]:
df_sw = df_sw.apply(lambda x: stopword_gen(x))
df_docs['sw'] = df_sw

In [ ]:
df_docs['sw']

category
Ground Beef             \s*grnd\s*|\s*gd\s*|\s*beef\s*|\s*bf\s*|\s*gro...
Top Round               \s*beef\s*|\s*rnd\s*|\s*bf\s*|\s*round\s*|\s*r...
Hot Dog                 \s*hot\s*|\s*dog\s*|\s*sausages\s*|\s*ss\s*|\s...
Beef Strip Steak        \s*strip\s*|\s*sk\s*|\s*stk\s*|\s*steak\s*|\s*...
New York Strip          \s*strip\s*|\s*york\s*|\s*new\s*|\s*strp\s*|\s...
Flat Iron Steak         \s*flt\s*|\s*sk\s*|\s*stk\s*|\s*ft\s*|\s*steak...
Skirt Steak             \s*skirt\s*|\s*sk\s*|\s*stk\s*|\s*st\s*|\s*ste...
Flap Meat               \s*flap\s*|\s*mt\s*|\s*beef\s*|\s*flp\s*|\s*bf...
Ground Chuck            \s*chuck\s*|\s*grnd\s*|\s*gd\s*|\s*beef\s*|\s*...
Wild Boar               \s*br\s*|\s*boar\s*|\s*wd\s*|\s*wld\s*|\s*wild\s*
Top Sirloin             \s*sn\s*|\s*sirloin\s*|\s*beef\s*|\s*bf\s*|\s*...
Short Ribs              \s*st\s*|\s*shrt\s*|\s*beef\s*|\s*ribs\s*|\s*b...
Hanger Steak            \s*sk\s*|\s*stk\s*|\s*hngr\s*|\s*steak\s*|\s*b...
Chuck Roast             \s*ch

In [ ]:
# preprocess df_gb['doc']

def my_preprocess(text):
  text = re.sub('[^a-zA-Z0-9 %#\./,:\-]', '', text)
  return text

In [ ]:
df_docs['doc'] = df_docs['doc'].apply(lambda x: my_preprocess(x))

In [ ]:
# with open('test_l.txt', 'w') as fh:
#   for word in text.split():
#     fh.write(word+'\n')

In [ ]:
# remove stopwords from docs

df_docs['doc'] = df_docs.apply(lambda row: re.sub(row['sw'], ' ', row['doc']), axis=1)

In [ ]:
# HOLD ON USE. Idea is to take new stopwords, get fuzzy matches from doc, and add those to stopwords... I think my regex mods will do fine.

# scores = {}
# for s in sw:
#   scores[s] = {}
#   for item in to_match:
#     score = fuzz.ratio(s, item)
#     if score < 75: continue
#     scores[s][item] = score

# scores 

In [ ]:
df_docs['doc']

category
Ground Beef             cab  chck fine 80/20 f whit open price urkey 8...
Top Round               96oz black shallow bowl bbrlcls corned btm flt...
Hot Dog                 vital fiber 7.76oz 10oz vegware compostable co...
Beef Strip Steak          loin 0x1 cho 180 gr pty 3-1 avg 75/25 export...
New York Strip          installed receptacles for cannons above bar em...
Flat Iron Steak         bread cuban 12 unsl loaf bkd leaf pasley 4oz b...
Skirt Steak             marinated eak wholesale thai chilli- grey gard...
Flap Meat                liver swift nacho  gltn free base  first chop...
Ground Chuck             chuk tendr mo angs chuk teres maj cho 114f 6/...
Wild Boar               tara shirako/cod male parts jpn fresh -hokkaid...
Top Sirloin             hens rock cornish fresh flat thermometer-36 de...
Short Ribs              chkn thighs 70z cryo wagyu a5 b/i rip eak end ...
Hanger Steak            tenderloin bone-in cho ribeye hanging tender r...
Chuck Roast             strbu

In [ ]:
# TFIDF given NO (additional stopwords as already processed out) stopwords and default preprocess/tokenizer (already preprocessed)

token_pattern1 = r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b"
token_pattern2 = r'[\.\-#%/:]*[a-zA-Z0-9]+[\./#%\-:]*[a-zA-Z0-9\.#%/:]*'
token_pattern3 = r'^\b[a-z]+[\.\-%#:/]*[a-z]+\b|[0-9]+[\.\-%#:/][0-9]+[\w#]+|[a-z0-9]+[a-z#]+'

tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern=token_pattern3, ngram_range=(1, 2))
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_docs['doc'])

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()

df1 = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=feature_names, columns=test_cats)
df1['total'] = df1.sum(axis=1)
df1 = df1[(df1.index.str.len() >= 3) & (df1['total'] > 0)]

In [ ]:
df1.sort_values('Ground Beef', ascending=False)

,Ground Beef,Top Round,Hot Dog,Beef Strip Steak,New York Strip,Flat Iron Steak,Skirt Steak,Flap Meat,Ground Chuck,Wild Boar,Top Sirloin,Short Ribs,Hanger Steak,Chuck Roast,Filet Mignon,Tri Tip,Pork Rack,Chuck Roll,Prime Rib,Baby Back Ribs,Pork Head,Bone Marrow,T-Bone Steaks,Chuck Flat,Spare Ribs,Hot Link,Ham Hocks,Pork Skin,Pork Chops,Pork Neck Bones,Wagyu Beef Rib Eye,Ground Kobe Beef,Lamb,Cornish Game Hens,Pork Chop Center Cut,total
81/19,0.293672,0.001588,0.000000,0.023002,0.000835,0.002316,0.005024,0.008465,0.044027,0.000700,0.000000,0.000000,0.000857,0.022661,0.007237,0.0,0.000000,0.009623,0.010149,0.000000,0.0,0.013164,0.0,0.003677,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.021734,0.000000,0.0,0.003634,0.472368
pty,0.282931,0.002531,0.000707,0.097741,0.000000,0.009139,0.000000,0.021412,0.293221,0.000000,0.001786,0.022169,0.000780,0.008254,0.000000,0.0,0.001089,0.004381,0.019803,0.001785,0.0,0.000000,0.0,0.002511,0.001263,0.0,0.0,0.0,0.004897,0.002561,0.026821,0.032984,0.000000,0.0,0.000000,0.838766
80/20,0.276564,0.004298,0.000000,0.056387,0.000695,0.011571,0.019304,0.018012,0.219407,0.000000,0.004083,0.003685,0.001427,0.009434,0.006026,0.0,0.000996,0.017026,0.007243,0.000000,0.0,0.004697,0.0,0.018370,0.005774,0.0,0.0,0.0,0.035821,0.002341,0.008175,0.063333,0.005371,0.0,0.000000,0.800039
fine raw,0.223633,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001835,0.013495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.238963
chub,0.210288,0.000000,0.000000,0.009601,0.000000,0.000000,0.001375,0.007811,0.041784,0.000831,0.001164,0.001750,0.000000,0.002688,0.000000,0.0,0.000000,0.008562,0.001720,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.002127,0.000000,0.000000,0.004297,0.000000,0.0,0.000000,0.293998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
denuded pan,0.000000,0.000000,0.000000,0.000000,0.000000,0.001833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.001833
denuded patty,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.001493
denuded philly,0.000000,0.000000,0.000000,0.000000,0.000000,0.001642,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001822,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.003463
denuded plate,0.000000,0.001885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.001885


In [ ]:
# d = {}
# for cat in df1.columns:
#   counts = df1[cat][df1[cat] > df1[cat].quantile(.95)].shape[0]
#   d[cat] = counts

# d

In [ ]:
# d = {}
# for keyword in top_idf.index:
#   count = sum(1 for match in re.finditer(r'\b%s\b' % keyword, text))
#   d[keyword] = count
# top_idf['counts'] = d.values()

# top_idf.sort_values('counts', ascending=False)

In [ ]:
df_temp = pd.DataFrame(columns=['keyword', 'tfidf', 'counts', 'category'])
for cat in df1.columns[0:-1]:
  # get all features with scores in top 50%
  top_idf = df1[cat][df1[cat] > df1[cat].quantile(.95)].to_frame()
  
  # get counts of features from df_docs
  d = {}
  text = df_docs.loc[cat, 'doc']
  for keyword in top_idf.index:
    count = len(re.findall(keyword, text))
    d[keyword] = count
  top_idf['counts'] = d.values()

  #add category column
  top_idf['category'] = cat

  #some sprucing up
  top_idf.reset_index(inplace=True)
  top_idf.rename(columns={'index':'keyword', cat:'tfidf'}, inplace=True)
  top_idf = top_idf[top_idf['counts'] > 25]

  df_temp = df_temp.append(top_idf)

df_temp.reset_index(drop=True, inplace=True)

In [ ]:
df_temp.to_csv('derp.csv')

<h1>Method 2: Global Stopwords from Categories </h1>

<h2> Stopwords </h2>

In [ ]:
# Modified stopwords generator function w/o regex characters

def stopword_gen2(stopwords):
  # sw generator
  new_sw = []
  new_sw = new_sw + stopwords #populate with base stopwords
  
  for word in stopwords:
    # new stopwords with all vowels removed
    if not re.match('^[aeiou].*|.*[aeiou]$', word) and len(word) > 3:
      no_v_word = re.sub('[aeiou]', '', word)
      new_sw.append(no_v_word)

    # first and last letter
    if not re.match('^[aeiou].*|.*[aeiou]$|^l.*b$|^k.*g$|^o.*z$|^g.*m$', word) and len(word) > 3:
      short_word = word[0] + word[-1]
      new_sw.append(short_word)

    # special for chicken
    if word == 'chicken': new_sw + ['chix', 'chx']

  return list(set(new_sw))  #modified return w/o regex chars

In [ ]:
# generate stopwords list
df_sw = df_sw.apply(lambda x: stopword_gen2(x))

In [ ]:
my_stopwords = list(set(df_sw.sum()))

<h2> Preprocessor </h2>
... same as above ...

In [ ]:
def my_preprocess(text):
  text = re.sub('[^a-zA-Z0-9 %#\.\/\:-]', '', text)
  return text

TFIDF Vectorizer Using custom stopwords list and preprocessor from above

In [ ]:
# tfidf using custom stopwords and preprocessor

tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern=r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b", ngram_range=(1, 2), stop_words=my_stopwords, preprocessor=my_preprocess)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_docs['doc'])

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()

df2 = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=feature_names, columns=test_cats)
df2['total'] = df2.sum(axis=1)
df2.sort_values(by='total', ascending=False)

Get top 50 keywords by cat

In [ ]:
df_temp2 = pd.DataFrame(columns=['keyword', 'tfidf', 'counts', 'category'])
for cat in df2.columns[0:-1]:
  # setup top 50 by category tfidf
  top_idf = df2[cat].nlargest(50).to_frame()
  
  # get counts of features from df_docs
  d = {}
  text = df_docs.loc[cat, 'doc']
  for keyword in top_idf.index:
    count = sum(1 for match in re.finditer(r'\b%s\b' % re.escape(keyword), text))
    d[keyword] = count
  top_idf['counts'] = d.values()

  #add category column
  top_idf['category'] = cat

  #some sprucing up
  top_idf.reset_index(inplace=True)
  top_idf.rename(columns={'index':'keyword', cat:'tfidf'}, inplace=True)

  df_temp2 = df_temp2.append(top_idf)

df_temp2.reset_index(drop=True, inplace=True)

In [ ]:
df_temp2.to_csv('tfidf_meats2.csv')


<h1> Method 2.a: Global Stopwords using high IDF Features </h1>

In [ ]:
# tfidf using custom stopwords and preprocessor

tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern=r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b", min_df=.80) # no ngram range as we're looking for unigram stopwords
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_docs['doc'])

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()

df3 = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=feature_names, columns=test_cats)
df3['total'] = df2.sum(axis=1)
df3.sort_values(by='total', ascending=False)

,Ground Beef,Top Round,Hot Dog,Beef Strip Steak,New York Strip,Flat Iron Steak,Skirt Steak,Flap Meat,Ground Chuck,Wild Boar,Top Sirloin,Short Ribs,Hanger Steak,Chuck Roast,Filet Mignon,Tri Tip,Pork Rack,Chuck Roll,Prime Rib,Baby Back Ribs,Pork Head,Bone Marrow,T-Bone Steaks,Chuck Flat,Spare Ribs,Hot Link,Ham Hocks,Pork Skin,Pork Chops,Pork Neck Bones,Wagyu Beef Rib Eye,Ground Kobe Beef,Lamb,Cornish Game Hens,Pork Chop Center Cut,total
oz,0.021507,0.104744,0.689279,0.131018,0.193552,0.082357,0.074327,0.023074,0.022811,0.110844,0.060475,0.038725,0.067363,0.056044,0.149672,0.022712,0.023830,0.012632,0.137727,0.003973,0.340158,0.029906,0.109966,0.349235,0.009694,0.053985,0.057478,0.091771,0.133539,0.011624,0.059396,0.037246,0.013019,0.264595,0.139468,5.298707
choice,0.016570,0.057790,0.044352,0.076375,0.203844,0.120335,0.125770,0.047675,0.014131,0.012600,0.072485,0.098346,0.270316,0.026972,0.212061,0.183228,0.001480,0.101531,0.062066,0.002776,0.003066,0.086734,0.080164,0.022822,0.004816,0.000000,0.000000,0.003108,0.000000,0.005197,0.069158,0.026494,0.000000,0.098586,0.005502,4.716353
bnls,0.017699,0.012408,0.000000,0.066184,0.075401,0.043998,0.033478,0.034962,0.041824,0.036759,0.021384,0.123590,0.041006,0.054998,0.113458,0.034349,0.020147,0.054272,0.044381,0.004319,0.000000,0.027237,0.031183,0.056067,0.053860,0.000000,0.868456,0.060458,0.115563,0.005054,0.056494,0.073613,0.042454,0.017431,0.082044,4.217528
cc,0.001478,0.002450,0.022061,0.105865,0.103414,0.019486,0.010488,0.005102,0.002674,0.015995,0.100137,0.021854,0.031952,0.015147,0.039121,0.004804,0.035424,0.025606,0.033104,0.004832,0.000000,0.064881,0.005815,0.002614,0.020961,0.000000,0.003495,0.000000,0.271213,0.000000,0.003010,0.000000,0.000000,0.000000,0.153660,3.002258
loin,0.006208,0.002190,0.000000,0.126975,0.080266,0.004583,0.009374,0.009500,0.004780,0.091898,0.012276,0.024583,0.005858,0.016076,0.039960,0.005367,0.070516,0.006539,0.015977,0.124160,0.017891,0.017572,0.005197,0.001168,0.086644,0.000000,0.012496,0.021160,0.128246,0.030326,0.021521,0.005521,0.016982,0.000000,0.117716,2.676177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
st,0.000296,0.002994,0.006894,0.012342,0.036739,0.002393,0.002497,0.001701,0.000000,0.079973,0.001329,0.010550,0.025398,0.000000,0.001597,0.008407,0.143305,0.009511,0.013904,0.074291,0.000000,0.001966,0.005815,0.001307,0.366814,0.000000,0.000000,0.040587,0.039421,0.002828,0.003010,0.000000,0.000000,0.009752,0.035920,NaN
steak,0.006650,0.011433,0.004136,0.145908,0.438148,0.364081,0.535897,0.006378,0.014707,0.006855,0.048296,0.060663,0.553015,0.011360,0.122155,0.033628,0.006441,0.011706,0.197301,0.000000,0.000000,0.052102,0.031982,0.006535,0.000000,0.000000,0.045439,0.003382,0.014191,0.000000,0.072239,0.014414,0.000000,0.000000,0.003991,NaN
stk,0.017882,0.008983,0.002758,0.211457,0.091168,0.291265,0.143339,0.019984,0.022061,0.011425,0.217110,0.022607,0.219568,0.006627,0.209180,0.031226,0.001610,0.023411,0.074153,0.000000,0.000000,0.007864,0.069779,0.003921,0.001310,0.000000,0.003495,0.003382,0.000000,0.000000,0.060199,0.043241,0.000000,0.000000,0.007982,NaN
strip,0.008620,0.003175,0.001340,0.448984,0.447100,0.044200,0.049037,0.023147,0.004549,0.017770,0.012922,0.030035,0.039026,0.000920,0.086928,0.015178,0.001565,0.017069,0.055996,0.000000,0.003243,0.022936,0.016958,0.001270,0.000000,0.000000,0.000000,0.032880,0.000000,0.000000,0.064373,0.020017,0.006157,0.009480,0.009700,NaN


In [ ]:
my_stopwords2 = df3.index.tolist()

In [ ]:
# tfidf using custom stopwords and preprocessor

tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern=r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b", ngram_range=(1,2), stop_words=my_stopwords2, preprocessor=my_preprocess)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_docs['doc'])

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()

df3 = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=feature_names, columns=test_cats)
df3['total'] = df2.sum(axis=1)
df3.sort_values(by='total', ascending=False)

,Ground Beef,Top Round,Hot Dog,Beef Strip Steak,New York Strip,Flat Iron Steak,Skirt Steak,Flap Meat,Ground Chuck,Wild Boar,Top Sirloin,Short Ribs,Hanger Steak,Chuck Roast,Filet Mignon,Tri Tip,Pork Rack,Chuck Roll,Prime Rib,Baby Back Ribs,Pork Head,Bone Marrow,T-Bone Steaks,Chuck Flat,Spare Ribs,Hot Link,Ham Hocks,Pork Skin,Pork Chops,Pork Neck Bones,Wagyu Beef Rib Eye,Ground Kobe Beef,Lamb,Cornish Game Hens,Pork Chop Center Cut,total
cab,0.102649,0.033189,0.000331,0.153080,0.019181,0.118406,0.055851,0.069935,0.136100,0.000696,0.097150,0.130775,0.090578,0.014940,0.011808,0.039360,0.000000,0.080628,0.134525,0.001886,0.0,0.001221,0.016169,0.044825,0.000000,0.0,0.000000,0.000000,0.004754,0.010749,0.027491,0.013186,0.000000,0.000000,0.002225,3.384487
angus,0.129454,0.031475,0.000511,0.160083,0.029268,0.060146,0.064307,0.033066,0.052931,0.000239,0.065631,0.100444,0.087087,0.054143,0.078429,0.037520,0.000000,0.061008,0.077658,0.000971,0.0,0.103074,0.007280,0.018764,0.000000,0.0,0.000000,0.000000,0.000000,0.009218,0.026875,0.031663,0.001723,0.000000,0.000000,3.247491
patty,0.168417,0.023420,0.000000,0.100166,0.002885,0.018133,0.063465,0.026110,0.398121,0.000000,0.001618,0.008268,0.002163,0.058087,0.009365,0.005422,0.000000,0.074999,0.071565,0.000000,0.0,0.015986,0.000000,0.021542,0.002491,0.0,0.000000,0.000000,0.001728,0.000000,0.065957,0.148601,0.000000,0.000928,0.004853,3.134548
louis,0.000000,0.000000,0.000000,0.000667,0.000000,0.000000,0.002520,0.000000,0.000000,0.004115,0.000000,0.035160,0.000000,0.000000,0.000000,0.000000,0.078283,0.001275,0.066161,0.181065,0.0,0.000000,0.000000,0.000000,0.508420,0.0,0.000000,0.024721,0.043460,0.000000,0.000000,0.000000,0.000000,0.000000,0.057432,2.733811
ribeye,0.032909,0.011373,0.000000,0.187563,0.009108,0.117413,0.031441,0.024692,0.007714,0.000000,0.052630,0.063237,0.078434,0.003440,0.014213,0.003072,0.001239,0.025956,0.116426,0.000000,0.0,0.015526,0.022479,0.000000,0.002419,0.0,0.001494,0.000000,0.011750,0.000000,0.087356,0.060526,0.000000,0.000000,0.025926,2.444704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ztf new,0.000000,0.000000,0.000000,0.000000,0.001028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
zucchini baby,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005865,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
zucchini brd,0.000000,0.000000,0.000000,0.001286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
zuckers heavy,0.000000,0.000000,0.000000,0.000000,0.000000,0.001567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [ ]:
df_temp3 = pd.DataFrame(columns=['keyword', 'tfidf', 'counts', 'category'])
for cat in df3.columns[0:-1]:
  # setup top 50 by category tfidf
  top_idf = df3[cat].nlargest(50).to_frame()
  
  # get counts of features from df_docs
  d = {}
  text = df_docs.loc[cat, 'doc']
  for keyword in top_idf.index:
    count = sum(1 for match in re.finditer(r'\b%s\b' % re.escape(keyword), text))
    d[keyword] = count
  top_idf['counts'] = d.values()

  #add category column
  top_idf['category'] = cat

  #some sprucing up
  top_idf.reset_index(inplace=True)
  top_idf.rename(columns={'index':'keyword', cat:'tfidf'}, inplace=True)

  df_temp3 = df_temp3.append(top_idf)

df_temp3.reset_index(drop=True, inplace=True)

In [ ]:
df_temp3.to_csv('tfidf_meats3.csv')